HW6_Q6_Programming Part: Multi-class and Multi-Label Classification Using Support Vector Machines- Hardik Prajapati(2678294168)

In [2]:
import os
import numpy as np
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.svm import SVC
from sklearn.metrics import hamming_loss
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings("ignore")
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE


In [3]:
df=pd.read_csv("../data/Anuran Calls (MFCCs)/Frogs_MFCCs.csv")
df

,MFCCs_ 1,MFCCs_ 2,MFCCs_ 3,MFCCs_ 4,MFCCs_ 5,MFCCs_ 6,MFCCs_ 7,MFCCs_ 8,MFCCs_ 9,MFCCs_10,...,MFCCs_17,MFCCs_18,MFCCs_19,MFCCs_20,MFCCs_21,MFCCs_22,Family,Genus,Species,RecordID
0,1.0,0.152936,-0.105586,0.200722,0.317201,0.260764,0.100945,-0.150063,-0.171128,0.124676,...,-0.108351,-0.077623,-0.009568,0.057684,0.118680,0.014038,Leptodactylidae,Adenomera,AdenomeraAndre,1
1,1.0,0.171534,-0.098975,0.268425,0.338672,0.268353,0.060835,-0.222475,-0.207693,0.170883,...,-0.090974,-0.056510,-0.035303,0.020140,0.082263,0.029056,Leptodactylidae,Adenomera,AdenomeraAndre,1
2,1.0,0.152317,-0.082973,0.287128,0.276014,0.189867,0.008714,-0.242234,-0.219153,0.232538,...,-0.050691,-0.023590,-0.066722,-0.025083,0.099108,0.077162,Leptodactylidae,Adenomera,AdenomeraAndre,1
3,1.0,0.224392,0.118985,0.329432,0.372088,0.361005,0.015501,-0.194347,-0.098181,0.270375,...,-0.136009,-0.177037,-0.130498,-0.054766,-0.018691,0.023954,Leptodactylidae,Adenomera,AdenomeraAndre,1
4,1.0,0.087817,-0.068345,0.306967,0.330923,0.249144,0.006884,-0.265423,-0.172700,0.266434,...,-0.048885,-0.053074,-0.088550,-0.031346,0.108610,0.079244,Leptodactylidae,Adenomera,AdenomeraAndre,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7190,1.0,-0.554504,-0.337717,0.035533,0.034511,0.443451,0.093889,-0.100753,0.037087,0.081075,...,0.069430,0.071001,0.021591,0.052449,-0.021860,-0.079860,Hylidae,Scinax,ScinaxRuber,60
7191,1.0,-0.517273,-0.370574,0.030673,0.068097,0.402890,0.096628,-0.116460,0.063727,0.089034,...,0.061127,0.068978,0.017745,0.046461,-0.015418,-0.101892,Hylidae,Scinax,ScinaxRuber,60
7192,1.0,-0.582557,-0.343237,0.029468,0.064179,0.385596,0.114905,-0.103317,0.070370,0.081317,...,0.082474,0.077771,-0.009688,0.027834,-0.000531,-0.080425,Hylidae,Scinax,ScinaxRuber,60
7193,1.0,-0.519497,-0.307553,-0.004922,0.072865,0.377131,0.086866,-0.115799,0.056979,0.089316,...,0.051796,0.069073,0.017963,0.041803,-0.027911,-0.096895,Hylidae,Scinax,ScinaxRuber,60


In [4]:
#Encode the labels
le = preprocessing.LabelEncoder()
label_family=le.fit_transform(df.iloc[:,22])
label_genus=le.fit_transform(df.iloc[:,23])
label_species=le.fit_transform(df.iloc[:,24])
df.iloc[:,22]=label_family
df.iloc[:,23]=label_genus
df.iloc[:,24]=label_species
df


,MFCCs_ 1,MFCCs_ 2,MFCCs_ 3,MFCCs_ 4,MFCCs_ 5,MFCCs_ 6,MFCCs_ 7,MFCCs_ 8,MFCCs_ 9,MFCCs_10,...,MFCCs_17,MFCCs_18,MFCCs_19,MFCCs_20,MFCCs_21,MFCCs_22,Family,Genus,Species,RecordID
0,1.0,0.152936,-0.105586,0.200722,0.317201,0.260764,0.100945,-0.150063,-0.171128,0.124676,...,-0.108351,-0.077623,-0.009568,0.057684,0.118680,0.014038,3,0,0,1
1,1.0,0.171534,-0.098975,0.268425,0.338672,0.268353,0.060835,-0.222475,-0.207693,0.170883,...,-0.090974,-0.056510,-0.035303,0.020140,0.082263,0.029056,3,0,0,1
2,1.0,0.152317,-0.082973,0.287128,0.276014,0.189867,0.008714,-0.242234,-0.219153,0.232538,...,-0.050691,-0.023590,-0.066722,-0.025083,0.099108,0.077162,3,0,0,1
3,1.0,0.224392,0.118985,0.329432,0.372088,0.361005,0.015501,-0.194347,-0.098181,0.270375,...,-0.136009,-0.177037,-0.130498,-0.054766,-0.018691,0.023954,3,0,0,1
4,1.0,0.087817,-0.068345,0.306967,0.330923,0.249144,0.006884,-0.265423,-0.172700,0.266434,...,-0.048885,-0.053074,-0.088550,-0.031346,0.108610,0.079244,3,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7190,1.0,-0.554504,-0.337717,0.035533,0.034511,0.443451,0.093889,-0.100753,0.037087,0.081075,...,0.069430,0.071001,0.021591,0.052449,-0.021860,-0.079860,2,7,9,60
7191,1.0,-0.517273,-0.370574,0.030673,0.068097,0.402890,0.096628,-0.116460,0.063727,0.089034,...,0.061127,0.068978,0.017745,0.046461,-0.015418,-0.101892,2,7,9,60
7192,1.0,-0.582557,-0.343237,0.029468,0.064179,0.385596,0.114905,-0.103317,0.070370,0.081317,...,0.082474,0.077771,-0.009688,0.027834,-0.000531,-0.080425,2,7,9,60
7193,1.0,-0.519497,-0.307553,-0.004922,0.072865,0.377131,0.086866,-0.115799,0.056979,0.089316,...,0.051796,0.069073,0.017963,0.041803,-0.027911,-0.096895,2,7,9,60


In [5]:
#split test-train data 
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,:22], df.iloc[:,22:25], test_size=0.30, random_state=42)
print("Training data is of length {} ".format(len(X_train)))
print("Testing data is of length {} ".format(len(X_test)))
X_train

Training data is of length 5036 
Testing data is of length 2159 


,MFCCs_ 1,MFCCs_ 2,MFCCs_ 3,MFCCs_ 4,MFCCs_ 5,MFCCs_ 6,MFCCs_ 7,MFCCs_ 8,MFCCs_ 9,MFCCs_10,...,MFCCs_13,MFCCs_14,MFCCs_15,MFCCs_16,MFCCs_17,MFCCs_18,MFCCs_19,MFCCs_20,MFCCs_21,MFCCs_22
2004,1.000000,0.327476,0.282440,0.572211,0.096509,-0.036705,-0.082174,0.146604,0.297338,-0.120902,...,0.438532,-0.148451,-0.260375,0.178628,0.218024,-0.042083,-0.126511,-0.090279,0.096176,0.191697
1194,1.000000,0.232818,-0.144331,-0.078909,-0.072467,0.255380,0.450215,0.261033,-0.168462,-0.281445,...,-0.108483,0.329984,0.274031,-0.157579,-0.291979,-0.042932,-0.009021,-0.032679,0.023912,0.041407
5359,0.966443,0.455914,1.000000,0.541650,-0.412873,0.307099,0.235819,-0.064025,0.069728,-0.052552,...,-0.073050,-0.140994,0.162584,-0.007981,-0.049428,-0.053166,-0.026728,0.029443,0.019032,-0.105221
1756,1.000000,0.257873,0.112807,0.527614,0.186094,0.043683,-0.163682,-0.014125,0.196527,0.018839,...,0.404813,-0.075488,-0.320882,0.024806,0.226971,0.034969,-0.106029,-0.147124,0.042627,0.250249
497,1.000000,0.401859,0.563219,0.578304,-0.126517,-0.058725,0.431806,0.066801,-0.321014,0.155569,...,-0.132244,0.190753,0.126674,-0.148049,-0.002891,0.157072,-0.015251,-0.055428,0.086927,0.086006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3772,1.000000,0.406366,0.268684,0.704863,0.358415,0.055376,-0.159812,0.065813,0.345068,0.175460,...,0.378497,-0.014091,-0.352985,0.015183,0.220504,0.054246,-0.069108,-0.174583,-0.032202,0.188263
5191,1.000000,0.634337,0.594395,0.403271,0.004960,0.037880,-0.065399,0.125276,0.106976,-0.223357,...,-0.228140,-0.274889,0.244973,0.096130,-0.140923,0.003646,0.007163,0.068718,0.047640,-0.179215
5226,1.000000,0.756413,0.732319,0.385807,-0.034821,0.073881,-0.058027,0.183775,0.219419,-0.445112,...,-0.177708,-0.311815,0.182382,0.030100,-0.182019,0.006296,-0.005868,0.029439,0.004783,-0.198014
5390,0.931410,0.626137,1.000000,0.375834,-0.223044,0.325943,0.121659,-0.092542,0.163365,-0.049382,...,-0.047610,-0.188463,0.102240,0.150353,-0.054750,-0.054434,0.108092,0.156994,-0.051681,-0.085656


In [57]:
#seperating the labels of test and train data 
y_train_family=y_train.iloc[:,0]
y_train_genus=y_train.iloc[:,1]
y_train_species=y_train.iloc[:,2]
y_test_family=y_test.iloc[:,0]
y_test_genus=y_test.iloc[:,1]
y_test_species=y_test.iloc[:,2]

In [58]:
def svm_optimalParam(model,xtrain,ytrain,xtest,ytest,parameters):
    classifier = GridSearchCV(estimator=model,**parameters)
    classifier.fit(xtrain,ytrain)
    average=classifier.cv_results_['mean_test_score']
    deviation=classifier.cv_results_['std_test_score']
    print("The grid scores of cross validation for coefficients are : ")
    for mean,std,para in zip(average, deviation,classifier.cv_results_['params']):
        print (' %0.3f (+/- %0.3f) : Coeff %r' %(mean,std,para))
    print(" .")
    print("The optimal coefficients are : {}".format(classifier.best_params_))
    predicty_test=classifier.predict(xtest)
    print(" ")
    print(classification_report(ytest,predicty_test))
    return classifier

coeff = {'C' : np.logspace(1, 4, 4), 'gamma' : np.logspace(-3, 6, 10)}
cv_svm = StratifiedKFold(10, random_state=5036, shuffle=True)
svm_gaussian_parameters = {
    'param_grid' : coeff, 
    'cv' : cv_svm,
    'scoring' : 'f1_weighted', 
    'verbose' : 1
}


In [59]:
svm_gaussian_clf ={}
#Label:Family - SVM (one vs all, Gaussian Kernel )
svm_gaussian_clf['Family']=svm_optimalParam(SVC(kernel='rbf',decision_function_shape='ovr'),X_train,y_train_family,X_test,y_test_family,svm_gaussian_parameters)

Fitting 10 folds for each of 40 candidates, totalling 400 fits
The grid scores of cross validation for coefficients are : 
 0.868 (+/- 0.012) : Coeff {'C': 10.0, 'gamma': 0.001}
 0.929 (+/- 0.014) : Coeff {'C': 10.0, 'gamma': 0.01}
 0.965 (+/- 0.009) : Coeff {'C': 10.0, 'gamma': 0.1}
 0.991 (+/- 0.004) : Coeff {'C': 10.0, 'gamma': 1.0}
 0.987 (+/- 0.004) : Coeff {'C': 10.0, 'gamma': 10.0}
 0.771 (+/- 0.021) : Coeff {'C': 10.0, 'gamma': 100.0}
 0.484 (+/- 0.007) : Coeff {'C': 10.0, 'gamma': 1000.0}
 0.464 (+/- 0.002) : Coeff {'C': 10.0, 'gamma': 10000.0}
 0.462 (+/- 0.001) : Coeff {'C': 10.0, 'gamma': 100000.0}
 0.462 (+/- 0.001) : Coeff {'C': 10.0, 'gamma': 1000000.0}
 0.927 (+/- 0.014) : Coeff {'C': 100.0, 'gamma': 0.001}
 0.946 (+/- 0.010) : Coeff {'C': 100.0, 'gamma': 0.01}
 0.984 (+/- 0.005) : Coeff {'C': 100.0, 'gamma': 0.1}
 0.992 (+/- 0.005) : Coeff {'C': 100.0, 'gamma': 1.0}
 0.987 (+/- 0.004) : Coeff {'C': 100.0, 'gamma': 10.0}
 0.771 (+/- 0.021) : Coeff {'C': 100.0, 'gamma': 

In [60]:
#Label:Genus - SVM (one vs all, Gaussian Kernel )
svm_gaussian_clf['Genus']=svm_optimalParam(SVC(kernel='rbf',decision_function_shape='ovr'),X_train,y_train_genus,X_test,y_test_genus,svm_gaussian_parameters)

Fitting 10 folds for each of 40 candidates, totalling 400 fits
The grid scores of cross validation for coefficients are : 
 0.766 (+/- 0.010) : Coeff {'C': 10.0, 'gamma': 0.001}
 0.918 (+/- 0.006) : Coeff {'C': 10.0, 'gamma': 0.01}
 0.974 (+/- 0.007) : Coeff {'C': 10.0, 'gamma': 0.1}
 0.988 (+/- 0.005) : Coeff {'C': 10.0, 'gamma': 1.0}
 0.980 (+/- 0.007) : Coeff {'C': 10.0, 'gamma': 10.0}
 0.716 (+/- 0.017) : Coeff {'C': 10.0, 'gamma': 100.0}
 0.438 (+/- 0.004) : Coeff {'C': 10.0, 'gamma': 1000.0}
 0.421 (+/- 0.001) : Coeff {'C': 10.0, 'gamma': 10000.0}
 0.421 (+/- 0.001) : Coeff {'C': 10.0, 'gamma': 100000.0}
 0.421 (+/- 0.001) : Coeff {'C': 10.0, 'gamma': 1000000.0}
 0.916 (+/- 0.006) : Coeff {'C': 100.0, 'gamma': 0.001}
 0.955 (+/- 0.008) : Coeff {'C': 100.0, 'gamma': 0.01}
 0.986 (+/- 0.007) : Coeff {'C': 100.0, 'gamma': 0.1}
 0.989 (+/- 0.005) : Coeff {'C': 100.0, 'gamma': 1.0}
 0.980 (+/- 0.007) : Coeff {'C': 100.0, 'gamma': 10.0}
 0.716 (+/- 0.017) : Coeff {'C': 100.0, 'gamma': 

In [61]:
#Label:Species - SVM (one vs all, Gaussian Kernel )
svm_gaussian_clf['Species']=svm_optimalParam(SVC(kernel='rbf',decision_function_shape='ovr'),X_train,y_train_species,X_test,y_test_species,svm_gaussian_parameters)

Fitting 10 folds for each of 40 candidates, totalling 400 fits
The grid scores of cross validation for coefficients are : 
 0.793 (+/- 0.007) : Coeff {'C': 10.0, 'gamma': 0.001}
 0.935 (+/- 0.011) : Coeff {'C': 10.0, 'gamma': 0.01}
 0.978 (+/- 0.008) : Coeff {'C': 10.0, 'gamma': 0.1}
 0.989 (+/- 0.005) : Coeff {'C': 10.0, 'gamma': 1.0}
 0.976 (+/- 0.009) : Coeff {'C': 10.0, 'gamma': 10.0}
 0.643 (+/- 0.017) : Coeff {'C': 10.0, 'gamma': 100.0}
 0.333 (+/- 0.005) : Coeff {'C': 10.0, 'gamma': 1000.0}
 0.318 (+/- 0.001) : Coeff {'C': 10.0, 'gamma': 10000.0}
 0.318 (+/- 0.001) : Coeff {'C': 10.0, 'gamma': 100000.0}
 0.318 (+/- 0.001) : Coeff {'C': 10.0, 'gamma': 1000000.0}
 0.934 (+/- 0.011) : Coeff {'C': 100.0, 'gamma': 0.001}
 0.974 (+/- 0.008) : Coeff {'C': 100.0, 'gamma': 0.01}
 0.986 (+/- 0.006) : Coeff {'C': 100.0, 'gamma': 0.1}
 0.988 (+/- 0.006) : Coeff {'C': 100.0, 'gamma': 1.0}
 0.976 (+/- 0.009) : Coeff {'C': 100.0, 'gamma': 10.0}
 0.643 (+/- 0.017) : Coeff {'C': 100.0, 'gamma': 

In [63]:
def multilabel_summary(xtest, ytrue, classifier_type, clf_title):
    predict_Y_df = pd.DataFrame(columns=ytrue.columns)
    for col in ytrue.columns:
        clf = classifier_type[col]
        ypredict = clf.predict(xtest)
        predict_Y_df.loc[:, col] = ypredict
    print("Multilabel evaluation of:  {} category".format(clf_title))
    print("-" * 80)
    hamming, exact_ratio = metric_score(predict_Y_df,ytrue)
    return [hamming, exact_ratio]
    

def metric_score(predictY,groundtruth_Y):
    misclassification = 0
    for truth, pred in zip(groundtruth_Y.values, predictY.values):
        miss_cost = (truth != pred)
        misclassification += np.sum(miss_cost)
    hamming = misclassification / (groundtruth_Y.shape[0] * groundtruth_Y.shape[1])
    
    exact_ratio = 0
    for truth, pred in zip(groundtruth_Y.values, predictY.values):
        match_score = (truth == pred)
        if sum(match_score) == groundtruth_Y.shape[1]:
            exact_ratio += 1;
    exact_ratio /= groundtruth_Y.shape[0]
    
    hamming = np.round(hamming, 4)
    exact_ratio = np.round(exact_ratio, 4)
    print("Hamming Loss for these classifiers is {}".format(hamming))
    print("Exact-Match Ratio for these classifiers is {}".format(exact_ratio))
    return hamming, exact_ratio

In [64]:
classifier_summary ={}
classifier_summary['SVM_ova_GaussianKernel_nonStandardized'] = multilabel_summary(X_test, y_test,svm_gaussian_clf,'Gaussian Kernel SVM OneVsAll w/o Standarization')


Multilabel evaluation of:  Gaussian Kernel SVM OneVsAll w/o Standarization category
--------------------------------------------------------------------------------
Hamming Loss for these classifiers is 0.0085
Exact-Match Ratio for these classifiers is 0.9861


Q6_b_iii : Repeat 6(b)ii with L1-penalized SVMs. Remember to normalize the at-
tributes.

In [69]:
#standardizing data
sc = StandardScaler()
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.fit_transform(X_test)

In [81]:
coeff = {'C' : np.logspace(1, 5, 10)}
svm_L1_parameters = {
    'param_grid' : coeff, 
    'cv' : cv_svm,
    'scoring' : 'f1_weighted', 
    'verbose' : 1
}
svm_L1_clf ={}
#Label:Family - SVM (L1)
svm_L1_clf['Family']=svm_optimalParam(LinearSVC(penalty='l1',dual=False),X_train_std,y_train_family,X_test_std,y_test_family,svm_L1_parameters)


Fitting 10 folds for each of 10 candidates, totalling 100 fits
The grid scores of cross validation for coefficients are : 
 0.937 (+/- 0.010) : Coeff {'C': 10.0}
 0.937 (+/- 0.010) : Coeff {'C': 27.825594022071243}
 0.937 (+/- 0.010) : Coeff {'C': 77.4263682681127}
 0.937 (+/- 0.010) : Coeff {'C': 215.44346900318823}
 0.937 (+/- 0.010) : Coeff {'C': 599.4842503189409}
 0.937 (+/- 0.010) : Coeff {'C': 1668.100537200059}
 0.937 (+/- 0.010) : Coeff {'C': 4641.588833612777}
 0.937 (+/- 0.010) : Coeff {'C': 12915.496650148827}
 0.937 (+/- 0.010) : Coeff {'C': 35938.13663804626}
 0.937 (+/- 0.010) : Coeff {'C': 100000.0}
 .
The optimal coefficients are : {'C': 10.0}
 
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        27
           1       0.86      0.88      0.87       162
           2       0.88      0.91      0.89       623
           3       0.96      0.96      0.96      1347

    accuracy                           0.93      2159
   

In [82]:
#Label:Genus - SVM (L1)
svm_L1_clf['Genus']=svm_optimalParam(LinearSVC(penalty='l1',dual=False),X_train_std,y_train_genus,X_test_std,y_test_genus,svm_L1_parameters)


Fitting 10 folds for each of 10 candidates, totalling 100 fits
The grid scores of cross validation for coefficients are : 
 0.947 (+/- 0.009) : Coeff {'C': 10.0}
 0.947 (+/- 0.009) : Coeff {'C': 27.825594022071243}
 0.947 (+/- 0.009) : Coeff {'C': 77.4263682681127}
 0.947 (+/- 0.009) : Coeff {'C': 215.44346900318823}
 0.947 (+/- 0.009) : Coeff {'C': 599.4842503189409}
 0.947 (+/- 0.009) : Coeff {'C': 1668.100537200059}
 0.947 (+/- 0.009) : Coeff {'C': 4641.588833612777}
 0.947 (+/- 0.009) : Coeff {'C': 12915.496650148827}
 0.947 (+/- 0.009) : Coeff {'C': 35938.13663804626}
 0.947 (+/- 0.009) : Coeff {'C': 100000.0}
 .
The optimal coefficients are : {'C': 10.0}
 
              precision    recall  f1-score   support

           0       0.96      0.98      0.97      1251
           1       0.90      0.93      0.91       162
           2       0.89      0.70      0.79        84
           3       0.90      0.97      0.93       468
           4       0.99      0.89      0.93        96
    

In [83]:
#Label:Species - SVM (L1)
svm_L1_clf['Species']=svm_optimalParam(LinearSVC(penalty='l1',dual=False),X_train_std,y_train_species,X_test_std,y_test_species,svm_L1_parameters)


Fitting 10 folds for each of 10 candidates, totalling 100 fits
The grid scores of cross validation for coefficients are : 
 0.960 (+/- 0.008) : Coeff {'C': 10.0}
 0.960 (+/- 0.008) : Coeff {'C': 27.825594022071243}
 0.959 (+/- 0.008) : Coeff {'C': 77.4263682681127}
 0.959 (+/- 0.008) : Coeff {'C': 215.44346900318823}
 0.959 (+/- 0.008) : Coeff {'C': 599.4842503189409}
 0.959 (+/- 0.008) : Coeff {'C': 1668.100537200059}
 0.959 (+/- 0.008) : Coeff {'C': 4641.588833612777}
 0.959 (+/- 0.008) : Coeff {'C': 12915.496650148827}
 0.959 (+/- 0.008) : Coeff {'C': 35938.13663804626}
 0.959 (+/- 0.008) : Coeff {'C': 100000.0}
 .
The optimal coefficients are : {'C': 10.0}
 
              precision    recall  f1-score   support

           0       0.96      0.91      0.93       220
           1       0.99      1.00      0.99      1031
           2       0.90      0.96      0.93       162
           3       0.92      0.77      0.84        84
           4       0.93      0.96      0.95       135
    

In [84]:
classifier_summary['SVM_L1_Standardized'] = multilabel_summary(X_test_std, y_test,svm_L1_clf,'SVM L1 penalized with Standarized data')


Multilabel evaluation of:  SVM L1 penalized with Standarized data category
--------------------------------------------------------------------------------
Hamming Loss for these classifiers is 0.0587
Exact-Match Ratio for these classifiers is 0.9092


Q6_b_iv : Repeat 6(b)iii by using SMOTE or any other method you know to remedy
class imbalance. Report your conclusions about the classifiers you trained.

In [88]:
def svm_optimalParam_smote(model,xtrain_sm,ytrain_sm,xtest_sm,ytest_sm,parameters_sm):
    clf_sm = Pipeline([
        ('sampling', SMOTE()),
        ('classification', model)
    ])
    best_model = svm_optimalParam(clf_sm,xtrain_sm,ytrain_sm,xtest_sm,ytest_sm,parameters_sm )
    return best_model

coeff = {'classification__C' : np.logspace(1, 5, 10)}
svm_smote_parameters = {
    'param_grid' : coeff, 
    'cv' : cv_svm,
    'scoring' : 'f1_weighted', 
    'verbose' : 1
}
svm_smote_clf ={}


In [89]:
#Label:Family - SVM (smote)
svm_smote_clf['Family']=svm_optimalParam_smote(LinearSVC(penalty='l1',dual=False),X_train_std,y_train_family,X_test_std,y_test_family,svm_smote_parameters)


Fitting 10 folds for each of 10 candidates, totalling 100 fits
The grid scores of cross validation for coefficients are : 
 0.928 (+/- 0.009) : Coeff {'classification__C': 10.0}
 0.929 (+/- 0.010) : Coeff {'classification__C': 27.825594022071243}
 0.930 (+/- 0.010) : Coeff {'classification__C': 77.4263682681127}
 0.930 (+/- 0.011) : Coeff {'classification__C': 215.44346900318823}
 0.929 (+/- 0.011) : Coeff {'classification__C': 599.4842503189409}
 0.929 (+/- 0.009) : Coeff {'classification__C': 1668.100537200059}
 0.930 (+/- 0.010) : Coeff {'classification__C': 4641.588833612777}
 0.928 (+/- 0.010) : Coeff {'classification__C': 12915.496650148827}
 0.929 (+/- 0.011) : Coeff {'classification__C': 35938.13663804626}
 0.928 (+/- 0.011) : Coeff {'classification__C': 100000.0}
 .
The optimal coefficients are : {'classification__C': 215.44346900318823}
 
              precision    recall  f1-score   support

           0       0.35      0.89      0.51        27
           1       0.75      0

In [90]:
#Label:Genus - SVM (smote)
svm_smote_clf['Genus']=svm_optimalParam_smote(LinearSVC(penalty='l1',dual=False),X_train_std,y_train_genus,X_test_std,y_test_genus,svm_smote_parameters)


Fitting 10 folds for each of 10 candidates, totalling 100 fits
The grid scores of cross validation for coefficients are : 
 0.921 (+/- 0.011) : Coeff {'classification__C': 10.0}
 0.921 (+/- 0.011) : Coeff {'classification__C': 27.825594022071243}
 0.919 (+/- 0.011) : Coeff {'classification__C': 77.4263682681127}
 0.922 (+/- 0.010) : Coeff {'classification__C': 215.44346900318823}
 0.922 (+/- 0.011) : Coeff {'classification__C': 599.4842503189409}
 0.922 (+/- 0.011) : Coeff {'classification__C': 1668.100537200059}
 0.920 (+/- 0.010) : Coeff {'classification__C': 4641.588833612777}
 0.918 (+/- 0.011) : Coeff {'classification__C': 12915.496650148827}
 0.921 (+/- 0.010) : Coeff {'classification__C': 35938.13663804626}
 0.921 (+/- 0.010) : Coeff {'classification__C': 100000.0}
 .
The optimal coefficients are : {'classification__C': 215.44346900318823}
 
              precision    recall  f1-score   support

           0       0.98      0.90      0.94      1251
           1       0.84      0

In [91]:
#Label:Species - SVM (smote)
svm_smote_clf['Species']=svm_optimalParam_smote(LinearSVC(penalty='l1',dual=False),X_train_std,y_train_species,X_test_std,y_test_species,svm_smote_parameters)


Fitting 10 folds for each of 10 candidates, totalling 100 fits
The grid scores of cross validation for coefficients are : 
 0.958 (+/- 0.008) : Coeff {'classification__C': 10.0}
 0.958 (+/- 0.008) : Coeff {'classification__C': 27.825594022071243}
 0.957 (+/- 0.007) : Coeff {'classification__C': 77.4263682681127}
 0.959 (+/- 0.008) : Coeff {'classification__C': 215.44346900318823}
 0.958 (+/- 0.008) : Coeff {'classification__C': 599.4842503189409}
 0.957 (+/- 0.008) : Coeff {'classification__C': 1668.100537200059}
 0.957 (+/- 0.009) : Coeff {'classification__C': 4641.588833612777}
 0.958 (+/- 0.008) : Coeff {'classification__C': 12915.496650148827}
 0.955 (+/- 0.009) : Coeff {'classification__C': 35938.13663804626}
 0.957 (+/- 0.007) : Coeff {'classification__C': 100000.0}
 .
The optimal coefficients are : {'classification__C': 215.44346900318823}
 
              precision    recall  f1-score   support

           0       0.98      0.87      0.92       220
           1       1.00      0

In [92]:
classifier_summary['SVM_smote_Standardized'] = multilabel_summary(X_test_std, y_test,svm_smote_clf,'SVM L1 penalized + SMOTE with standardised data')


Multilabel evaluation of:  SVM L1 penalized + SMOTE with standardised data category
--------------------------------------------------------------------------------
Hamming Loss for these classifiers is 0.0735
Exact-Match Ratio for these classifiers is 0.8657


In [93]:
classifier_summary['Loss-Metric'] = ['Hamming Loss', 'Exact match ratio']
final_summary = pd.DataFrame.from_dict(classifier_summary)
final_summary.set_index('Loss-Metric', inplace=True)
final_summary

,SVM_ova_GaussianKernel_nonStandardized,SVM_L1_Standardized,SVM_smote_Standardized
Loss-Metric,,,
Hamming Loss,0.0085,0.0587,0.0735
Exact match ratio,0.9861,0.9092,0.8657
